In [1]:
!pip install finance-datareader
!pip install opendartreader
!pip install --upgrade opendartreader

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# 주식가격 로딩
import FinanceDataReader as fdr

# 재무재표 로딩
import OpenDartReader

import datetime

In [3]:
start_date = '20180101'
end_date = datetime.datetime.now().strftime("%Y%m%d")
sample_code = '005930'
stock = fdr.DataReader( sample_code, start = start_date, end = end_date )

In [23]:
def pro_equ(code, start, end):
    # OpenDartReader 사용
    api_key = '9b9157d4313a84f3141d73f879c5d239d49bb0ce'
    dart = OpenDartReader(api_key)
    arr = []
    
    for i in range(int(start[:4]), int(end[:4])):
        # corp: 종목코드, bsns_year: 사업연도, reprt_code: 보고서코드
        # (1분기보고서 : 11013, 반기보고서 : 11012, 3분기보고서 : 11014, 사업보고서 : 11011)
        fs_before = dart.finstate_all(corp = code, bsns_year= str(int(start[:4])-1), fs_div='CFS', reprt_code=11011) 
        fs_present = dart.finstate_all(corp = code, bsns_year= start[:4], fs_div='CFS', reprt_code=11014)

        # 자본, 부채는 당기 금액('thstrm_amount') 값을 통해서 구함
        equity = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount'].replace(",", "")) # 당기자본(자본총계)
        liability = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", "")) # 당기부채(부채총계)

        # 자산 총계
        assets = equity + liability
        
        # 2019 4분기 ~ 2020 3분기까지의 당기순이익의 합을 구하려면 2019년 4분기 당기순이익과 2020년 1분기 ~ 3분기 당기순이익의 합을 알아여함
        # 2020년 1분기 ~ 3분기 당기순이익의 합은 2020년 3분기 손익계산서에서 'thstrm_add_amount' 값을 가져오면 되고
        # 2019년 4분기 당기순이익은 2019년 전체 당기순이익에서 2019년 1분기 ~ 3분기 당기순이익의 합을 빼서 구할 수 있음
        
        profit_before_3Q = int(fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",", "")) # 당기순이익
        profit_before = int(fs_before.loc[fs_before['sj_div'].isin(['IS']) & fs_before['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount'].replace(",", "")) # 당기순이익
        profit_present = int(fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",", "")) # 당기순이익
        profit = (profit_before-profit_before_3Q) + profit_present
        profit
        
        arr = arr + [(equity, liability, assets, profit)]
        
        return arr

In [47]:
arr =pro_equ('005930','2020','2021')

In [43]:
arr

[(276136188000000, 99652554000000, 375788742000000, 24873372000000)]

In [31]:
fs_before = dart.finstate_all(corp = '005930', bsns_year= '2017', fs_div='CFS', reprt_code=11011) 
fs_present = dart.finstate_all(corp = '005930', bsns_year= '2018', fs_div='CFS', reprt_code=11014)

In [32]:
fs_before

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,thstrm_add_amount
0,20180402005019,11011,2017,00126380,BS,재무상태표,ifrs_CurrentAssets,유동자산,-,제 49 기,146982464000000,제 48 기,141429704000000,제 47 기,124814725000000,1,NaN
1,20180402005019,11011,2017,00126380,BS,재무상태표,ifrs_CashAndCashEquivalents,현금및현금성자산,-,제 49 기,30545130000000,제 48 기,32111442000000,제 47 기,22636744000000,2,NaN
2,20180402005019,11011,2017,00126380,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 49 기,49447696000000,제 48 기,52432411000000,제 47 기,44228800000000,3,NaN
3,20180402005019,11011,2017,00126380,BS,재무상태표,-표준계정코드 미사용-,단기매도가능금융자산,-,제 49 기,3191375000000,제 48 기,3638460000000,제 47 기,4627530000000,4,NaN
4,20180402005019,11011,2017,00126380,BS,재무상태표,dart_ShortTermTradeReceivable,매출채권,-,제 49 기,27695995000000,제 48 기,24279211000000,제 47 기,25168026000000,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,20180402005019,11011,2017,00126380,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|지배기업 소유주지분|기타자본항목,제 49 기,-13899191000000,제 48 기,-11934586000000,제 47 기,-17580451000000,20,NaN
193,20180402005019,11011,2017,00126380,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|지배기업 소유주지분|매각예정분류기타자본항목,제 49 기,,제 48 기,-28810000000,제 47 기,23797000000,20,NaN
194,20180402005019,11011,2017,00126380,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|지배기업 소유주지분|이익잉여금 [member],제 49 기,215811200000000,제 48 기,193086317000000,제 47 기,185132014000000,20,NaN
195,20180402005019,11011,2017,00126380,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|지배기업 소유주지분|자본금 [member],제 49 기,897514000000,제 48 기,897514000000,제 47 기,897514000000,20,NaN


In [33]:
fs_present

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
0,20181114001530,11014,2018,00126380,BS,재무상태표,ifrs_CurrentAssets,유동자산,-,제 50 기 3분기말,176282049000000,제 49 기말,146982464000000,1,NaN,NaN,NaN,NaN
1,20181114001530,11014,2018,00126380,BS,재무상태표,ifrs_CashAndCashEquivalents,현금및현금성자산,-,제 50 기 3분기말,33088093000000,제 49 기말,30545130000000,2,NaN,NaN,NaN,NaN
2,20181114001530,11014,2018,00126380,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 50 기 3분기말,58681418000000,제 49 기말,49447696000000,3,NaN,NaN,NaN,NaN
3,20181114001530,11014,2018,00126380,BS,재무상태표,-표준계정코드 미사용-,단기매도가능금융자산,-,제 50 기 3분기말,,제 49 기말,3191375000000,4,NaN,NaN,NaN,NaN
4,20181114001530,11014,2018,00126380,BS,재무상태표,-표준계정코드 미사용-,단기상각후원가금융자산,-,제 50 기 3분기말,3446114000000,제 49 기말,,5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,20181114001530,11014,2018,00126380,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|지배기업 소유주지분|기타자본항목,제 50 기 3분기,-12453349000000,NaN,NaN,18,NaN,제 49 기 3분기,-8525520000000,NaN
201,20181114001530,11014,2018,00126380,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|지배기업 소유주지분|매각예정분류기타자본항목,제 50 기 3분기,,NaN,NaN,18,NaN,제 49 기 3분기,-44482000000,NaN
202,20181114001530,11014,2018,00126380,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|지배기업 소유주지분|이익잉여금 [member],제 50 기 3분기,241628322000000,NaN,NaN,18,NaN,제 49 기 3분기,206773005000000,NaN
203,20181114001530,11014,2018,00126380,SCE,자본변동표,ifrs_Equity,기말자본,자본 [member]|지배기업 소유주지분|자본금 [member],제 50 기 3분기,897514000000,NaN,NaN,18,NaN,제 49 기 3분기,897514000000,NaN


In [35]:
int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount'].replace(",", ""))

TypeError: cannot convert the series to <class 'int'>

In [40]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity'])]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount


In [41]:
fs_present['account_id'].isin(['ifrs-full_Equity'])

0      False
1      False
2      False
3      False
4      False
       ...  
200    False
201    False
202    False
203    False
204    False
Name: account_id, Length: 205, dtype: bool